In [5]:
import os
import json
import base64
from openai import OpenAI
from dotenv import load_dotenv
from pathlib import Path

# Load environment variables
load_dotenv()

# Initialize OpenAI client with API key from .env
api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    raise ValueError("Please set OPENAI_API_KEY in your .env file")
client = OpenAI(api_key=api_key)

# Path to image folder and output file
image_folder = Path("pixel_data")
output_file = image_folder / "descriptions.jsonl"

# Get all jpg files
image_files = list(image_folder.glob("*.jpg"))

# Process each image
for image_path in image_files:
    try:
        # Encode image to base64
        with open(image_path, 'rb') as image_file:
            base64_image = base64.b64encode(image_file.read()).decode('utf-8')
        
        # Generate description using GPT-4 Vision
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "Describe this pixel art image in detail, and keep all description short and concise — no more than 25 words."},
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}"
                            }
                        }
                    ]
                }
            ],
            max_tokens=300
        )
        
        # Create entry
        entry = {
            "file_name": image_path.name,
            "prompt": response.choices[0].message.content
        }
        
        # Write to output file
        with open(output_file, 'a') as f:
            json.dump(entry, f)
            f.write('\n')
            
        print(f"Processed {image_path.name}")
        print(f"Description: {entry['prompt']}")
        
    except Exception as e:
        print(f"Error processing {image_path.name}: {str(e)}")

print("Processing complete!")


Processed pixel_17.jpg
Description: A chubby, gray pixel art creature with a simple face, small beak, and green border, resembling a cartoonish bird or animal.
Processed pixel_03.jpg
Description: A cute pink pixel heart with a minimalist facial expression, centered on a black background, conveying a sense of friendliness and charm.
Processed pixel_02.jpg
Description: A square button with a blue square and a speech bubble icon, set against a dark gray background, in pixel art style.
Processed pixel_16.jpg
Description: A simplistic black creature with large eyes, a red mouth, and a quirky expression, set against a blank white background.
Processed pixel_14.jpg
Description: A pixel art torch with a brown wooden handle, bright red flames, and a yellow core, set against a dark blue background.
Processed pixel_28.jpg
Description: A blue pixelated thumbs-up icon on a white background, featuring distinct pixel edges and a simplified hand structure, conveying a positive gesture.
Processed pixel